In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mlelec.data.dataset import precomputed_molecules, MoleculeDataset, MLDataset
import torch
from ase.io import read
import ase
from mlelec.models.linear import LinearTargetModel

In [3]:
h_data = MoleculeDataset(mol_name='water_1000', frame_slice=slice(0,10), data_path = 'examples/data/water_1000/sto-3g', aux_path = 'examples/data/water_1000/sto-3g', device='cuda', aux=['overlap', 'orbitals']) #frames =frames, frame_slice=':4', target_data={'fock': h}, aux=['overlap', 'orbitals'],aux_data = {'overlap': over, 'orbitals':orbs }
h_ml = MLDataset(molecule_data=h_data, device ='cuda')

Loading structures
examples/data/water_1000/sto-3g/fock.hickle


In [4]:
# h_ml._shuffle(random_seed=5381)
# h_ml._split_indices(train_frac=0.7, val_frac=0.2)

In [5]:
for f in h_ml.structures:
    f.pbc = False

## training on a tiny dataset for now 

In [6]:
linmod = LinearTargetModel(dataset = h_ml, metrics = "l2_loss", nlayers = 1, nout = 1, nhidden = 10, bias = False, device = 'cuda')


Computing features with default hypers
cuda:0 cuda
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=False)
    (1): Linear(in_features=10, out_features=1, bias=False)
  )
)
cuda:0 cuda
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=False)
    (1): Linear(in_features=10, out_features=1, bias=False)
  )
)
cuda:0 cuda
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=384, out_features=10, bias=False)
    (1): Linear(in_features=10, out_features=1, bias=False)
  )
)
cuda:0 cuda
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=False)
    (1): Linear(in_features=10, out_features=1, bias=False)
  )
)
cuda:0 cuda
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=384, out_features=10, bias=False)
    (1): Linear(in_features=10, out_features=1, bias=False)
  )
)
cuda:0 cuda
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=False)
    (1): Linear(in_features=10,

In [7]:
optimizer = torch.optim.Adam(linmod.parameters(), lr=0.01)
for epoch in range(100):
    loss = linmod.forward()
    loss.backward()
    optimizer.step()

    if epoch%10 == 0:
        print(torch.sqrt(loss.detach()))

cuda:0 cuda:0
tensor(69.3503, device='cuda:0', dtype=torch.float64)
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
tensor(64.6939, device='cuda:0', dtype=torch.float64)
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
tensor(56.2616, device='cuda:0', dtype=torch.float64)
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
tensor(46.7579, device='cuda:0', dtype=torch.float64)
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
tensor(39.9948, device='cuda:0', dtype=torch.float64)
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
cuda:0 cuda:0
tensor(23.1046, 

The default features and model is quite bad - so no wonder losses are high

## Predictions

In [8]:
linmod.forward()
fock = linmod.reconstructed_tensor
print(fock.shape)
print(fock.dtype)

cuda:0 cuda:0
torch.Size([10, 7, 7])
torch.float32


In [9]:
fock = fock.type(torch.float64)

## plugging in predicted fock matrix into pyscfad

In [10]:

import os
os.environ['PYSCFAD_BACKEND']='torch'

import torch
from pyscf import gto

from pyscfad import numpy as np
from pyscfad import ops
from pyscfad.ml.scf import hf
import pyscf.pbc.tools.pyscf_ase as pyscf_ase


Using PyTorch backend.


/home/nigam/miniconda3/lib/python3.11/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [11]:

mol = gto.Mole()
mol.atom = pyscf_ase.ase_atoms_to_pyscf(h_ml.structures[0])
mol.basis = 'sto-3g'
mol.build()
fock = linmod.reconstructed_tensor[0].type(torch.float64)


mf = hf.SCF(mol)

mo_energy, mo_coeff = mf.eig(fock, s = torch.eye(fock.shape[-1], dtype = fock.dtype))
mo_occ = mf.get_occ(mo_energy) # get_occ returns a numpy array
mo_occ = ops.convert_to_tensor(mo_occ)


In [12]:
fock.dtype

torch.float64

In [19]:
fock[0]

tensor([-24.6125,  -2.7584,  -0.0690,   0.0000,  -0.0888,  -1.3858,  -1.5692],
       device='cuda:0', dtype=torch.float64, grad_fn=<SelectBackward0>)

In [13]:

dm1 = mf.make_rdm1(mo_coeff, mo_occ)
dip = mf.dip_moment(dm=dm1)
dip_norm = np.linalg.norm(dip)
dip_norm.backward(retain_graph=True)
print(fock.grad)

Dipole moment(X, Y, Z, Debye): -1.81266, -2.04481,  1.76728
None


/tmp/ipykernel_267180/1939595782.py:5: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(fock.grad)


In [14]:
dip_norm

tensor(3.2543, device='cuda:0', dtype=torch.float64,
       grad_fn=<LinalgVectorNormBackward0>)

In [15]:
dip_norm.backward()


In [17]:
fock.backward()

RuntimeError: grad can be implicitly created only for scalar outputs